# 1. Energy data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'N' ; opts.Ynorm = 'N' ;
opts.Ms = 100 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel100', 'models','t_dGP_train')

Linesearch     25;  Value 2.964163e+04


partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel30');
models = s.models;


## 1.1 RBCM vs TERBCM

In [5]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 1.78290054, MSLL 0.5679, NLPD 3.4270


### 1.1.4 Find multi q

In [6]:
criterion = 'TERBCM';

In [7]:
printf('Optimizing q\r\n');
iqs = 2.2 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.2064
   2.2095
   2.2059
   2.2070
   2.2082
   2.2056
   2.2050
   2.2049
   2.2006
   2.2047
   2.2013
   2.2056
   2.2063
   2.2049
   2.2085
   2.2077
   2.2017
   2.2012
   2.2077
   2.2060
   2.2066
   2.2030
   2.2017
   2.2027
   2.2027
   2.2082
   2.2031
   2.2050
   2.2038
   2.2013
   2.2016
   2.2090
   2.2079
   2.2092
   2.2048
   2.2064
   2.2054
   2.2046
   2.2007
   2.2056
   2.2005
   2.2047
   2.2012
   2.2041
   2.2014
   2.2083
   2.2002
   2.2043
   2.2035
   2.2076
   2.2018
   2.2009
   2.2094
   2.2047
   2.2079
   2.2096
   2.2059
   2.2065
   2.2066
   2.2071
   2.2054
   2.2049
   2.2004
   2.2031
   2.2072
   2.2092
   2.2019
   2.2004
   2.2002
   2.2081
   2.2062
   2.2069
   2.2005
   2.2016
   2.2091
   2.2013
   2.2048
   2.2040
   2.2089
   2.2012
   2.2037
   2.2081
   2.2082
   2.2053
   2.2099
   2.2051
   2.2097
   2.2020
   2.2036
   2.2086
   2.2078
   2.2007
   2.2076
   2.2013
   2.2033
   2.2058
   2.2056
   2.2065


In [8]:
printf('Optimizing q\r\n');
iqs = 0.4 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   0.40527
   0.40126
   0.40420
   0.40464
   0.40745
   0.40420
   0.40969
   0.40965
   0.40499
   0.40713
   0.40238
   0.40092
   0.40726
   0.40802
   0.40761
   0.40333
   0.40448
   0.40456
   0.40621
   0.40833
   0.40453
   0.40809
   0.40251
   0.40020
   0.40716
   0.40060
   0.40602
   0.40851
   0.40502
   0.40197
   0.40692
   0.40423
   0.40007
   0.40710
   0.40517
   0.40970
   0.40432
   0.40986
   0.40233
   0.40617
   0.40316
   0.40125
   0.40750
   0.40864
   0.40277
   0.40607
   0.40005
   0.40215
   0.40767
   0.40201
   0.40867
   0.40112
   0.40527
   0.40565
   0.40694
   0.40829
   0.40535
   0.40996
   0.40623
   0.40898
   0.40445
   0.40551
   0.40706
   0.40350
   0.40738
   0.40984
   0.40183
   0.40991
   0.40309
   0.40845
   0.40854
   0.40072
   0.40151
   0.40299
   0.40228
   0.40168
   0.40543
   0.40868
   0.40278
   0.40574
   0.40181
   0.40700
   0.40611
   0.40050
   0.40033
   0.40928
   0.40374
   0.40993
   0.40989
 

## 1.2 GRBCM vs TEGRBCM

In [9]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 1.73787921, MSLL -0.8087, NLPD 2.0503


### 1.2.4 Find multi q

In [10]:
criterion = 'TEGRBCM';

In [11]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.0064
   2.0029
   2.0030
   2.0075
   2.0004
   2.0095
   2.0002
   2.0074
   2.0093
   2.0004
   2.0085
   2.0045
   2.0073
   2.0080
   2.0061
   2.0092
   2.0048
   2.0056
   2.0061
   2.0003
   2.0039
   2.0024
   2.0082
   2.0011
   2.0077
   2.0037
   2.0072
   2.0075
   2.0015
   2.0069
   2.0076
   2.0045
   2.0038
   2.0056
   2.0027
   2.0041
   2.0058
   2.0072
   2.0077
   2.0088
   2.0056
   2.0029
   2.0003
   2.0099
   2.0036
   2.0025
   2.0043
   2.0019
   2.0074
   2.0100
   2.0094
   2.0061
   2.0049
   2.0022
   2.0001
   2.0053
   2.0074
   2.0021
   2.0016
   2.0077
   2.0002
   2.0062
   2.0015
   2.0065
   2.0048
   2.0025
   2.0005
   2.0027
   2.0026
   2.0048
   2.0070
   2.0099
   2.0034
   2.0054
   2.0000
   2.0022
   2.0034
   2.0098
   2.0005
   2.0098
   2.0032
   2.0085
   2.0068
   2.0090
   2.0077
   2.0039
   2.0050
   2.0048
   2.0068
   2.0010
   2.0028
   2.0084
   2.0067
   2.0061
   2.0042
   2.0041
   2.0025
   2.0074


In [12]:
printf('Optimizing q\r\n');
iqs = 0.2 + 0.01 * rand(opts.Ms,1)
ld = 100
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   0.20292
   0.20191
   0.20956
   0.20018
   0.20080
   0.20401
   0.20494
   0.20025
   0.20730
   0.20222
   0.20437
   0.20702
   0.20457
   0.20511
   0.20495
   0.20993
   0.20706
   0.20930
   0.20239
   0.20400
   0.20331
   0.20183
   0.20571
   0.20929
   0.20288
   0.20795
   0.20075
   0.20239
   0.20840
   0.20087
   0.20832
   0.20685
   0.20074
   0.20670
   0.20864
   0.20996
   0.20346
   0.20209
   0.20522
   0.20669
   0.20255
   0.20201
   0.20488
   0.20968
   0.20562
   0.20327
   0.20863
   0.20412
   0.20239
   0.20790
   0.20483
   0.20514
   0.20254
   0.20729
   0.20277
   0.20025
   0.20079
   0.20737
   0.20185
   0.20949
   0.20272
   0.20106
   0.20240
   0.20326
   0.20807
   0.20954
   0.20923
   0.20607
   0.20478
   0.20332
   0.20760
   0.20093
   0.20145
   0.20444
   0.20258
   0.20569
   0.20608
   0.20535
   0.20874
   0.20693
   0.20255
   0.20769
   0.20280
   0.20334
   0.20985
   0.20220
   0.20957
   0.20644
   0.20084
 

## 1.3 GPoE vs TEGPoE

In [13]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 1.45033641, MSLL 0.6716, NLPD 3.5306


### 1.3.4 Find multi q

In [14]:
criterion = 'TEGPoE';

In [15]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
ld = 10
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   2.0081
   2.0093
   2.0056
   2.0004
   2.0013
   2.0072
   2.0011
   2.0079
   2.0038
   2.0095
   2.0001
   2.0081
   2.0058
   2.0059
   2.0057
   2.0016
   2.0038
   2.0074
   2.0012
   2.0062
   2.0063
   2.0013
   2.0041
   2.0073
   2.0043
   2.0042
   2.0039
   2.0082
   2.0021
   2.0078
   2.0084
   2.0022
   2.0076
   2.0040
   2.0061
   2.0053
   2.0100
   2.0018
   2.0024
   2.0095
   2.0080
   2.0069
   2.0092
   2.0041
   2.0038
   2.0077
   2.0026
   2.0018
   2.0003
   2.0046
   2.0030
   2.0071
   2.0079
   2.0083
   2.0015
   2.0007
   2.0035
   2.0047
   2.0010
   2.0050
   2.0001
   2.0065
   2.0067
   2.0030
   2.0093
   2.0097
   2.0089
   2.0032
   2.0059
   2.0086
   2.0075
   2.0015
   2.0075
   2.0070
   2.0019
   2.0021
   2.0013
   2.0051
   2.0090
   2.0043
   2.0024
   2.0088
   2.0069
   2.0024
   2.0058
   2.0052
   2.0043
   2.0014
   2.0092
   2.0023
   2.0043
   2.0073
   2.0046
   2.0058
   2.0071
   2.0041
   2.0082
   2.0082


In [16]:
printf('Optimizing q\r\n');
iqs = 1.8 + 0.01 * rand(opts.Ms,1)
ld = 10
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
iqs =

   1.8035
   1.8047
   1.8048
   1.8044
   1.8075
   1.8082
   1.8001
   1.8045
   1.8037
   1.8083
   1.8077
   1.8059
   1.8054
   1.8089
   1.8038
   1.8084
   1.8076
   1.8076
   1.8093
   1.8073
   1.8063
   1.8050
   1.8025
   1.8037
   1.8005
   1.8069
   1.8085
   1.8095
   1.8006
   1.8093
   1.8086
   1.8041
   1.8086
   1.8018
   1.8063
   1.8054
   1.8017
   1.8001
   1.8041
   1.8097
   1.8023
   1.8090
   1.8014
   1.8032
   1.8007
   1.8065
   1.8082
   1.8005
   1.8099
   1.8088
   1.8018
   1.8063
   1.8092
   1.8061
   1.8071
   1.8053
   1.8019
   1.8090
   1.8084
   1.8099
   1.8067
   1.8074
   1.8079
   1.8073
   1.8022
   1.8069
   1.8055
   1.8065
   1.8088
   1.8099
   1.8039
   1.8013
   1.8002
   1.8070
   1.8002
   1.8099
   1.8068
   1.8074
   1.8081
   1.8052
   1.8066
   1.8003
   1.8026
   1.8048
   1.8038
   1.8023
   1.8020
   1.8012
   1.8028
   1.8008
   1.8088
   1.8058
   1.8016
   1.8023
   1.8073
   1.8012
   1.8030
   1.8090
